# EDA of Infogroup Datasets
#### We use this data

In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import clean
import glob

In [201]:
info_2022 = pd.read_csv("../data/raw/infogroup/poultry_plants_2022.csv")
print("This dataframe has", len(info_2022), "rows")
print("The column data types of the variables in this DF are\n", info_2022.dtypes)

This dataframe has 720 rows
The column data types of the variables in this DF are
 Unnamed: 0                        int64
COMPANY                          object
ADDRESS LINE 1                   object
CITY                             object
STATE                            object
ZIPCODE                         float64
ZIP4                            float64
COUNTY CODE                     float64
AREA CODE                         int64
IDCODE                            int64
LOCATION EMPLOYEE SIZE CODE      object
LOCATION SALES VOLUME CODE       object
PRIMARY SIC CODE                  int64
SIC6_DESCRIPTIONS                object
PRIMARY NAICS CODE              float64
NAICS8 DESCRIPTIONS              object
SIC CODE                        float64
SIC6_DESCRIPTIONS (SIC)          object
SIC CODE 1                      float64
SIC6_DESCRIPTIONS (SIC1)         object
SIC CODE 2                      float64
SIC6_DESCRIPTIONS(SIC2)          object
SIC CODE 3                      float

## To clean the infogroup dataset, we:
#### Made a large dataframe that contained each individual year's dataset, from 1997 to 2022, standardized column names to be all uppercase, turned NaNs to 0s, and turned float64 categorical variables (such as YEAR ESTABLISHED) into int64s

In [4]:
import clean

clean.clean_infogroup("../data/raw/infogroup/poultry_plants_2021.csv")
df = pd.read_csv("../data/clean/cleaned_infogroup_plants_all_time.csv")
df.head()

OSError: Cannot save file into a non-existent directory: 'data/clean'